In [41]:
# Importing the libraries to use
import numpy as np
import pandas as pd
import folium
import geojson
from geojson import FeatureCollection, Feature
from branca.colormap import linear

In [42]:
# Importing the Vodafone DataSet from a CSV file
dfVodafone = pd.read_csv("resources/data/June2023.csv")

/var/folders/vz/8vlfj1x9573f3y4tww42mzr40000gn/T/ipykernel_54610/1771366191.py:2: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  dfVodafone = pd.read_csv("resources/data/June2023.csv")


In [43]:
# Displaying the information about the dataframe columns
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173474 entries, 0 to 10173473
Data columns (total 25 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   Grid_ID     int64  
 2   Datetime    object 
 3   C1          float64
 4   C2          float64
 5   C3          float64
 6   C4          float64
 7   C5          float64
 8   C6          float64
 9   C7          float64
 10  C8          float64
 11  C9          float64
 12  C10         float64
 13  C11         float64
 14  D1          object 
 15  E1          float64
 16  E2          float64
 17  E3          float64
 18  E4          int64  
 19  E5          int64  
 20  E6          object 
 21  E7          int64  
 22  E8          float64
 23  E9          int64  
 24  E10         float64
dtypes: float64(16), int64(6), object(3)
memory usage: 1.9+ GB


In [44]:
# Displaying simple descriptive metrics
dfVodafone.describe().transpose().apply(lambda s: s.apply('{0:.0f}'.format))

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,10173474,5086736,2936829,0,2543368,5086736,7630105,10173473
Grid_ID,10173474,1872,1081,1,936,1871,2808,3743
C1,10173474,231,315,0,47,124,287,24075
C2,10173474,34,82,0,3,10,30,5609
C3,10173474,153,223,0,27,76,185,16061
C4,10173474,23,56,0,0,6,20,3738
C5,10173474,127,181,0,23,67,158,14761
C6,10173474,126,168,0,28,71,158,11556
C7,10173474,20,48,0,0,6,18,3441
C8,10173474,18,43,0,2,6,17,2726


In [45]:
# Including objects in the description
dfVodafone.describe(include=['O']).transpose()

,count,unique,top,freq
Datetime,10173474,2718,2023-06-01T00:00:00.000Z,3743
D1,7766008,3388818,Netherlands,246780
E6,2633524,129093,android:Total Traffic,2303580


In [46]:
# Transforming all columns from C1 to C11
dfVodafone['C1'] = dfVodafone['C1'].apply(np.int64)
dfVodafone['C2'] = dfVodafone['C2'].apply(np.int64)
dfVodafone['C3'] = dfVodafone['C3'].apply(np.int64)
dfVodafone['C4'] = dfVodafone['C4'].apply(np.int64)
dfVodafone['C5'] = dfVodafone['C5'].apply(np.int64)
dfVodafone['C6'] = dfVodafone['C6'].apply(np.int64)
dfVodafone['C7'] = dfVodafone['C7'].apply(np.int64)
dfVodafone['C8'] = dfVodafone['C8'].apply(np.int64)
dfVodafone['C9'] = dfVodafone['C9'].apply(np.int64)
dfVodafone['C10'] = dfVodafone['C10'].apply(np.int64)
dfVodafone['C11'] = dfVodafone['C11'].apply(np.int64)

In [47]:
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173474 entries, 0 to 10173473
Data columns (total 25 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   Grid_ID     int64  
 2   Datetime    object 
 3   C1          int64  
 4   C2          int64  
 5   C3          int64  
 6   C4          int64  
 7   C5          int64  
 8   C6          int64  
 9   C7          int64  
 10  C8          int64  
 11  C9          int64  
 12  C10         int64  
 13  C11         int64  
 14  D1          object 
 15  E1          float64
 16  E2          float64
 17  E3          float64
 18  E4          int64  
 19  E5          int64  
 20  E6          object 
 21  E7          int64  
 22  E8          float64
 23  E9          int64  
 24  E10         float64
dtypes: float64(5), int64(17), object(3)
memory usage: 1.9+ GB


In [48]:
# Changing the data type to date and with the format we want
dfVodafone['Datetime'] = pd.to_datetime(dfVodafone['Datetime'], format="%Y-%m-%dT%H:%M:%S.%fZ")

In [49]:
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173474 entries, 0 to 10173473
Data columns (total 25 columns):
 #   Column      Dtype         
---  ------      -----         
 0   Unnamed: 0  int64         
 1   Grid_ID     int64         
 2   Datetime    datetime64[ns]
 3   C1          int64         
 4   C2          int64         
 5   C3          int64         
 6   C4          int64         
 7   C5          int64         
 8   C6          int64         
 9   C7          int64         
 10  C8          int64         
 11  C9          int64         
 12  C10         int64         
 13  C11         int64         
 14  D1          object        
 15  E1          float64       
 16  E2          float64       
 17  E3          float64       
 18  E4          int64         
 19  E5          int64         
 20  E6          object        
 21  E7          int64         
 22  E8          float64       
 23  E9          int64         
 24  E10         float64       
dtypes: datetime64[ns

In [50]:
# Removing columns that we don't care about
dfVodafone = dfVodafone[["Grid_ID", "Datetime", "C1", "C2", "D1"]]

In [51]:
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10173474 entries, 0 to 10173473
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Grid_ID   int64         
 1   Datetime  datetime64[ns]
 2   C1        int64         
 3   C2        int64         
 4   D1        object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 388.1+ MB


In [52]:
# Importing the Vodafone Metadata with information about the cells
dfGrids = pd.read_csv("resources/metadata/wktComplete.csv", encoding="ISO-8859-1")

In [53]:
dfGrids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   grelha_id    3743 non-null   int64  
 1   dicofre      3743 non-null   int64  
 2   entity_id    3743 non-null   object 
 3   entity_type  3743 non-null   object 
 4   freguesia    3743 non-null   object 
 5   freguesias   3743 non-null   object 
 6   grelha_x     3743 non-null   int64  
 7   grelha_y     3743 non-null   int64  
 8   latitude     3743 non-null   float64
 9   longitude    3743 non-null   float64
 10  nome         3743 non-null   object 
 11  objectid     3743 non-null   int64  
 12  position     3743 non-null   object 
 13  wkt          3743 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 409.5+ KB


In [54]:
# Removing columns that we don't care about
dfGrids = dfGrids[["grelha_id", "freguesia", "latitude", "longitude", "nome", "position"]]

In [55]:
dfGrids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   grelha_id  3743 non-null   int64  
 1   freguesia  3743 non-null   object 
 2   latitude   3743 non-null   float64
 3   longitude  3743 non-null   float64
 4   nome       3743 non-null   object 
 5   position   3743 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 175.6+ KB


In [56]:
# Filtering by night time
dfVodafone = dfVodafone.set_index('Datetime').between_time('02:00', '07:00').reset_index()

In [57]:
dfVodafone.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200884 entries, 0 to 2200883
Data columns (total 5 columns):
 #   Column    Dtype         
---  ------    -----         
 0   Datetime  datetime64[ns]
 1   Grid_ID   int64         
 2   C1        int64         
 3   C2        int64         
 4   D1        object        
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 84.0+ MB


In [58]:
# Grouping by Grid ID and Day, aggregating by the minimum number of terminals each night for every cell
dfSleepByDay = dfVodafone.assign(day=dfVodafone["Datetime"].dt.day)
dfSleepByDay = dfSleepByDay.groupby(["Grid_ID", "day"], as_index=False).agg(C2=("C2", "min"), Datetime=('Datetime', "min"))
dfSleepByDay.rename(columns={"C2": "Sleeping"}, inplace=True)

In [59]:
dfSleepByDay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104804 entries, 0 to 104803
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Grid_ID   104804 non-null  int64         
 1   day       104804 non-null  int32         
 2   Sleeping  104804 non-null  int64         
 3   Datetime  104804 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int32(1), int64(2)
memory usage: 2.8 MB


In [60]:
# Grouping by Grid ID, aggregating by the mean number of minimum of terminals each night
dfSleepMean = dfSleepByDay.groupby(["Grid_ID"])["Sleeping"].mean().reset_index()

In [61]:
dfSleepMean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Grid_ID   3743 non-null   int64  
 1   Sleeping  3743 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 58.6 KB


In [62]:
# Adding the information about the cells to our dataframe
dfSleepMean = dfSleepMean.merge(dfGrids, left_on='Grid_ID', right_on='grelha_id')

In [63]:
dfSleepMean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743 entries, 0 to 3742
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Grid_ID    3743 non-null   int64  
 1   Sleeping   3743 non-null   float64
 2   grelha_id  3743 non-null   int64  
 3   freguesia  3743 non-null   object 
 4   latitude   3743 non-null   float64
 5   longitude  3743 non-null   float64
 6   nome       3743 non-null   object 
 7   position   3743 non-null   object 
dtypes: float64(3), int64(2), object(3)
memory usage: 234.1+ KB


In [64]:
# Creating a linear color map
colormap = linear.YlOrRd_09.scale(
    dfSleepMean["Sleeping"].min(), dfSleepMean["Sleeping"].max()
)

colormap

In [65]:
Sleeping_dict = dfSleepMean.set_index("Grid_ID")["Sleeping"]
color_dict = {key: colormap(Sleeping_dict[key]) for key in Sleeping_dict.keys()}

In [66]:
# Saving a Heat Map
map = folium.Map(location=[38.743094, -9.145999], zoom_start=13)

features = []
for index, sleepByDay in dfSleepMean.iterrows():
    feature = Feature(
        geometry=geojson.loads(sleepByDay["position"].replace("'", '"'))[0],
        properties={
            "Grid_ID": sleepByDay["Grid_ID"],
            "name": sleepByDay["nome"],
            "freguesy": sleepByDay["freguesia"],
            "sleeping": sleepByDay["Sleeping"],
        }
    )
    features.append(feature)

feature_collection = FeatureCollection(features)

tooltip = folium.GeoJsonTooltip(
    fields=["name", "freguesy", "sleeping"],
    aliases=["Zona:", "Freguesia:", "A Dormir:"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    data=feature_collection,
    style_function=lambda feature: {
        "fillColor": color_dict[feature.properties["Grid_ID"]],
        "color": "black",
        "weight": 0,
        "fillOpacity": .5,
    },
    tooltip=tooltip
).add_to(map)

colormap.add_to(map)

map.save('folium_testing_map.html')

map

In [67]:
# Showing top 20 cells
dfTop = dfSleepMean.nlargest(20, "Sleeping")[["Grid_ID", "nome", "freguesia", "Sleeping"]]
dfTop

,Grid_ID,nome,freguesia,Sleeping
571,572,Baixa,Santa Maria Maior,847.285714
686,687,Praça da Figueira,Santa Maria Maior,791.678571
624,625,Bairro Alto,Misericórdia,756.642857
511,512,Camões,Misericórdia,645.607143
628,629,Baixa,Santa Maria Maior,626.535714
570,571,Baixa,Santa Maria Maior,598.428571
627,628,Rossio,Santa Maria Maior,582.392857
516,517,Baixa,Santa Maria Maior,558.357143
682,683,Bairro Alto,Misericórdia,557.750000
515,516,Baixa,Santa Maria Maior,477.750000


In [68]:
# Group by freguesy, aggregate by sum
dfSleepMeanFreg = dfSleepMean.groupby("freguesia")["Sleeping"].sum().reset_index()

In [69]:
dfSleepMeanFreg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   freguesia  24 non-null     object 
 1   Sleeping   24 non-null     float64
dtypes: float64(1), object(1)
memory usage: 512.0+ bytes


In [70]:
dfTopFreg = dfSleepMeanFreg.nlargest(20, "Sleeping")[["freguesia", "Sleeping"]]
dfTopFreg

,freguesia,Sleeping
20,Santa Maria Maior,12927.535714
4,Arroios,8543.000000
21,Santo António,7111.464286
5,Avenidas Novas,6857.178571
15,Misericórdia,6780.428571
12,Estrela,2997.142857
23,São Vicente,2906.678571
17,Parque das Nações,2287.321429
18,Penha de França,1898.928571
16,Olivais,1862.071429


In [71]:
# Grouping by Weekday, aggregating by the mean number of minimum of terminals each weekday
dfSleepByWeekday = dfSleepByDay.assign(Weekday=dfSleepByDay['Datetime'].dt.day_name())
dfSleepByWeekdayMean = dfSleepByWeekday.groupby(["Weekday"], as_index=False)["Sleeping"].mean()

In [72]:
dfSleepByWeekdayMean

,Weekday,Sleeping
0,Friday,18.008015
1,Monday,16.747885
2,Saturday,18.744056
3,Sunday,18.436370
4,Thursday,17.201176
5,Tuesday,16.710326
6,Wednesday,17.123965
